# devqubit: Getting Started

**devqubit** is an experiment tracker designed specifically for quantum computing workflows.

### Why devqubit?

Quantum experiments have unique challenges:

| Challenge | How devqubit helps |
|-----------|--------------------|
| **Shot noise** | TVD-aware comparison that distinguishes real changes from statistical noise |
| **Circuit artifacts** | Native support for Qiskit, Cirq, PennyLane, Braket circuit formats |
| **Calibration drift** | Baseline verification to catch hardware changes |
| **Reproducibility** | Content-addressed storage ensures exact artifact matching |
| **Collaboration** | Portable bundles for sharing complete experiments |

### Core Concepts

devqubit organizes your work into:

- **Runs**: A single experiment execution with parameters, metrics, and artifacts
- **Projects**: A collection of related runs (e.g., "vqe_hydrogen", "qml_classifier")
- **Groups**: Runs that belong together (e.g., a hyperparameter sweep)
- **Baselines**: Reference runs for verification and comparison
- **Bundles**: Portable packages for sharing runs

### Architecture Overview

devqubit has two main storage components:

```python
┌─────────────────────────────────────────────────────────────────────────────┐
│                      Workspace (default: ~/.devqubit)                       │
│                                                                             │
│  ┌─────────────────────────┐    ┌─────────────────────────────────────────┐ │
│  │        Registry         │    │              Object Store               │ │
│  │      (registry.db)      │    │               (objects/)                │ │
│  │                         │    │                                         │ │
│  │  • Run metadata         │    │  • ExecutionEnvelopes (UEC JSON)        │ │
│  │  • Params & Metrics     │    │  • Circuits (QASM, QPY, JSON)           │ │
│  │  • Tags                 │    │  • Count distributions                  │ │
│  │  • Artifact references ─────▶│  • Device snapshots                     │ │
│  │  • Baselines            │    │  • Custom artifacts                     │ │
│  │  • Groups               │    │                                         │ │
│  │                         │    │  Content-addressed by SHA-256 (deduped) │ │
│  └─────────────────────────┘    └─────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────────────────┘
```

**Registry**: SQLite database for fast queries (list runs, search, get baseline)  
**Object Store**: Content-addressed file storage (artifacts are deduplicated by hash)

### ExecutionEnvelope (UEC)

The **Uniform Execution Contract (UEC)** is devqubit's standardized format for capturing complete execution context. When you use `run.wrap()`, adapters automatically create an **ExecutionEnvelope** containing:

```python
┌────────────────────────────────────────────────────────────────────────────────────────────┐
│                                     ExecutionEnvelope                                      │
│                              (stored in Object Store as JSON)                              │
├────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                            │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  │
│  │   producer   │  │   program    │  │    device    │  │  execution   │  │    result    │  │
│  │              │  │              │  │              │  │              │  │              │  │
│  │ • adapter    │  │ • logical    │  │ • backend    │  │ • shots      │  │ • success    │  │
│  │ • sdk        │  │ • physical   │  │ • qubits     │  │ • job_ids    │  │ • status     │  │
│  │ • versions   │  │ • hashes     │  │ • topology   │  │ • timestamps │  │ • counts     │  │
│  │              │  │              │  │ • calibration│  │ • options    │  │ • errors     │  │
│  └──────────────┘  └──────────────┘  └──────────────┘  └──────────────┘  └──────────────┘  │
│                           │                   │                                 │          │
│                           └───────────────────┴─────────────────────────────────┘          │
│                                          ArtifactRefs → Object Store                       │
└────────────────────────────────────────────────────────────────────────────────────────────┘
```

### Why Envelopes Matter

| Use Case | How Envelope Helps |
|----------|-------------------|
| **Reproducibility** | Captures exact circuit, device state, and settings |
| **Comparison** | Structural hashes enable fast program matching |
| **Verification** | TVD computed from normalized result counts |
| **Debugging** | Complete context when something goes wrong |
| **Calibration tracking** | Device snapshot includes calibration data |

This is why `run.wrap()` is powerful - it gives devqubit everything needed for robust comparison and verification.


### What you'll learn in this notebook:
1. **Configuration**: Set up your workspace
2. **Tracking**: Log parameters, metrics, tags, and artifacts
3. **Inspection**: Query and load runs from the registry
4. **Grouping**: Organize related runs (sweeps, experiments)
5. **Comparison**: Diff runs to see what changed
6. **Verification**: Check candidates against baselines
7. **Bundling**: Package runs for sharing

### 1. Configuration & Setup

devqubit needs to know where to store data. You have three options:

#### Option A: Default location
```python
# Uses ~/.devqubit automatically
from devqubit import track
with track(project="my_project") as run:
    ...
```

#### Option B: Environment variable
```bash
export DEVQUBIT_HOME=/path/to/workspace
```

#### Option C: Programmatic configuration
```python
from devqubit import Config, set_config
set_config(Config(root_dir="/path/to/workspace"))
```

For this tutorial, we'll use Option C to create an isolated demo workspace.

In [ ]:
from pathlib import Path
import shutil
import numpy as np

# Core devqubit imports
from devqubit import (
    track,  # Main context manager for runs
    Config,  # Configuration container
    set_config,  # Set global configuration
)

# Runs navigation and baseline management
from devqubit.runs import (
    load_run,
    list_runs,
    list_groups,
    list_runs_in_group,
    set_baseline,
    get_baseline,
)

# Storage functions
from devqubit.storage import create_store, create_registry

# Bundle utilities
from devqubit.bundle import pack_run, unpack_bundle, Bundle

# Comparison and verification
from devqubit.compare import diff, verify_baseline, VerifyPolicy

In [ ]:
# Create an isolated demo workspace
WORKSPACE = Path(".devqubit_getting_started")
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)

# Set global config — all devqubit calls will use this workspace
set_config(Config(root_dir=WORKSPACE))

np.random.seed(42)

print(f"Workspace created: {WORKSPACE.resolve()}")
print(f"Registry: {WORKSPACE / 'registry.db'}")
print(f"Objects:  {WORKSPACE / 'objects'}")

### 2. Creating Your First Run

The `track()` context manager is your main entry point. It:

1. Creates a new run with a unique ID
2. Provides methods to log data
3. Automatically finalizes the run when the block exits
4. Handles errors gracefully (marks run as failed)

#### What you can log:

| Method | Purpose | Example |
|--------|---------|--------|
| `log_param(key, value)` | Single parameter | `run.log_param("lr", 0.01)` |
| `log_params(dict)` | Multiple parameters | `run.log_params({"lr": 0.01, "epochs": 100})` |
| `log_metric(key, value)` | Single metric | `run.log_metric("accuracy", 0.95)` |
| `log_metric(key, value, step=n)` | Time-series metric | `run.log_metric("loss", 0.5, step=10)` |
| `log_metrics(dict)` | Multiple metrics | `run.log_metrics({"acc": 0.95, "f1": 0.92})` |
| `set_tag(key, value)` | Single tag | `run.set_tag("env", "production")` |
| `set_tags(dict)` | Multiple tags | `run.set_tags({"env": "prod", "version": "v2"})` |
| `log_json(name, obj)` | JSON artifact | `run.log_json("config", {"model": "cnn"})` |
| `log_text(name, text)` | Text artifact | `run.log_text("notes", "Experiment worked!")` |
| `log_binary(name, data)` | Binary artifact | `run.log_binary("weights", bytes_data)` |

#### Parameters vs Metrics vs Tags

- **Parameters**: Configuration that defines the experiment (inputs)
- **Metrics**: Results you measure (outputs)
- **Tags**: Labels for organization and filtering (metadata)

In [ ]:
# A simple mock experiment (simulates an optimization)
def mock_optimization(lr: float, iterations: int, seed: int) -> list[float]:
    """Simulate optimization that returns loss history."""
    rng = np.random.default_rng(seed)
    loss = 1.0
    history = []
    for _ in range(iterations):
        # Loss decreases with some randomness
        loss *= 1 - lr * rng.uniform(0.8, 1.0)
        history.append(float(loss))
    return history

In [ ]:
PROJECT = "optimization_demo"

with track(project=PROJECT, run_name="my_first_run") as run:

    # 1. Log parameters (the inputs to your experiment)
    run.log_params(
        {
            "learning_rate": 0.10,
            "iterations": 50,
            "seed": 42,
            "optimizer": "sgd",
        }
    )

    # 2. Set tags (for organization)
    run.set_tags(
        {
            "environment": "development",
            "role": "baseline",
            "author": "tutorial",
        }
    )

    # 3. Run the experiment
    history = mock_optimization(lr=0.10, iterations=50, seed=42)

    # 4. Log time-series metrics (one value per step)
    for step, loss in enumerate(history):
        run.log_metric("loss", loss, step=step)

    # 5. Log summary metrics (final values)
    run.log_metrics(
        {
            "final_loss": history[-1],
            "best_loss": min(history),
            "convergence_step": history.index(min(history)),
        }
    )

    # 6. Log artifacts (structured data)
    run.log_json(
        "full_config",
        {
            "lr": 0.10,
            "iterations": 50,
            "optimizer": "sgd",
            "description": "Baseline optimization run",
        },
        role="config",
    )

    run.log_json(
        "loss_history",
        {
            "values": history,
            "length": len(history),
        },
        role="result",
    )

    # Save run ID for later use
    first_run_id = run.run_id

print("Run completed!")
print(f"  Run ID: {first_run_id}")

#### Understanding Run IDs

Every run gets a unique ID (UUID). This ID:
- Is globally unique across all workspaces
- Never changes after creation
- Can be used to load, compare, or reference the run later

You can also provide a human-readable `run_name` for easier identification.

### 3. Inspecting Runs

After a run completes, you can load it from the registry to inspect its data.

The `RunRecord` object contains:
- `run_id`: Unique identifier
- `project`: Project name
- `run_name`: Human-readable name (optional)
- `status`: "completed", "failed", or "running"
- `params`: Dictionary of parameters
- `metrics`: Dictionary of metrics
- `tags`: Dictionary of tags
- `artifacts`: List of artifact references
- `started_at`, `ended_at`: Timestamps

In [ ]:
# Load the run we just created
record = load_run(first_run_id)

print("Run Details")
print("=" * 50)
print(f"Run ID:   {record.run_id}")
print(f"Name:     {record.run_name}")
print(f"Project:  {record.project}")
print(f"Status:   {record.status}")
print(f"Started:  {record.created_at}")
print(f"Ended:    {record.ended_at}")

In [ ]:
print("\nParameters:")
for key, value in record.params.items():
    print(f"  {key}: {value}")

In [ ]:
print("\nMetrics:")
for key, value in record.metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.6f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
print("\nTags:")
for key, value in record.tags.items():
    print(f"  {key}: {value}")

In [ ]:
print("\nArtifacts:")
for artifact in record.artifacts:
    print(f"    Role: {artifact.role}")
    print(f"    Digest: {artifact.digest[:24]}...\n")

### 4. Listing and Querying Runs

The registry provides methods to find runs:

| Method | Purpose |
|--------|--------|
| `list_runs(project=...)` | All runs in a project |
| `list_runs(limit=10)` | Recent runs (with limit) |
| `list_groups()` | All experiment groups |
| `list_runs_in_group(group_id)` | Runs in a specific group |
| `get_baseline(project)` | Current baseline for project |

In [ ]:
# List all runs in our project
print("Runs in project:")
for run_info in list_runs(project=PROJECT):
    print(f"  {run_info['run_id'][:12]}...  ({run_info.get('run_name', 'unnamed')})")

### 5. Grouped Runs (Sweeps & Experiments)

When running hyperparameter sweeps or related experiments, use **groups** to organize them:

```python
with track(
    project="my_project",
    group_id="sweep_001",        # Unique identifier for the group
    group_name="Learning Rate Sweep",  # Human-readable name
) as run:
    ...
```

Benefits:
- Query all runs in a group together
- Compare runs within a sweep
- Track which runs belong to which experiment

In [ ]:
# Run a learning rate sweep
learning_rates = [0.05, 0.10, 0.15, 0.20]
sweep_results = []

print("Running Learning Rate Sweep")
print("=" * 40)

for lr in learning_rates:
    with track(
        project=PROJECT,
        group_id="lr_sweep_001",  # All runs share this group ID
        group_name="Learning Rate Sweep",  # Human-readable name
    ) as run:
        # Log parameters
        run.log_params(
            {
                "learning_rate": lr,
                "iterations": 50,
                "seed": 42,
            }
        )
        run.set_tag("role", "sweep")

        # Run experiment
        history = mock_optimization(lr=lr, iterations=50, seed=42)

        # Log results
        run.log_metrics(
            {
                "final_loss": history[-1],
                "best_loss": min(history),
            }
        )

        sweep_results.append(
            {
                "lr": lr,
                "final_loss": history[-1],
                "run_id": run.run_id,
            }
        )

        print(f"  lr={lr:.2f}  =>  loss={history[-1]:.6f}")

# Find the best
best = min(sweep_results, key=lambda x: x["final_loss"])
print(f"\nBest: lr={best['lr']:.2f} (loss={best['final_loss']:.6f})")

In [ ]:
# Query groups
print("Experiment Groups:")
for group in list_groups():
    print(f"\n  {group['group_name']}")
    print(f"  ID: {group['group_id']}")

    # List runs in this group
    runs = list_runs_in_group(group["group_id"])
    print(f"  Runs: {len(runs)}")

    for run_info in runs:
        rec = load_run(run_info["run_id"])
        lr = rec.params.get("learning_rate", "?")
        loss = rec.metrics.get("final_loss", "?")
        print(f"    lr={lr}  loss={loss:.6f}")

### 6. Comparing Runs

The `diff()` function compares two runs and shows:

- **Parameter differences**: What configuration changed
- **Metric differences**: How results differ
- **Tag differences**: Metadata changes
- **TVD**: Distribution distance (for quantum count artifacts)

This is essential for understanding:
- Why one run performed better than another
- What changed between experiments
- Whether a new run is statistically similar to a baseline

In [ ]:
# Create a candidate run with different parameters
with track(project=PROJECT, run_name="candidate") as run:
    run.log_params(
        {
            "learning_rate": 0.12,  # Different from baseline (0.10)
            "iterations": 50,
            "seed": 42,
            "optimizer": "adam",  # Different optimizer
        }
    )
    run.set_tags({"role": "candidate", "environment": "development"})

    history = mock_optimization(lr=0.12, iterations=50, seed=42)

    run.log_metrics(
        {
            "final_loss": history[-1],
            "best_loss": min(history),
        }
    )

    candidate_id = run.run_id

print(f"Candidate run: {candidate_id}")

In [ ]:
# Compare baseline vs candidate
comparison = diff(first_run_id, candidate_id)
print(comparison)

### Interpreting the Diff

The diff shows:
- `<` values from the first run (baseline)
- `>` values from the second run (candidate)
- Changes in parameters, metrics, and other fields

For quantum experiments with count artifacts, you'd also see **TVD (Total Variation Distance)** — a measure of how different the probability distributions are.

### Understanding TVD (Total Variation Distance)

When comparing quantum measurement results, **TVD** quantifies how different two probability distributions are:

$\text{TVD}(P, Q) = \frac{1}{2} \sum_x |P(x) - Q(x)|$

**Interpretation:**
- **TVD = 0**: Distributions are identical
- **TVD = 0.01-0.05**: Typically shot noise — expected variation
- **TVD = 0.05-0.15**: May indicate real change (investigate)
- **TVD > 0.15**: Likely significant difference (code, calibration, or hardware)

**Why this matters for quantum:**

Quantum measurements are inherently probabilistic. Running the same circuit twice gives different counts due to shot noise. TVD helps distinguish:

| Scenario | Expected TVD | Action |
|----------|-------------|--------|
| Same circuit, same backend, different seeds | Low (0.01-0.03) | Normal, pass |
| Same circuit, different calibration | Medium (0.05-0.10) | Warning, investigate |
| Different circuit or bug | High (0.15+) | Fail, investigate |

devqubit uses TVD-aware verification to avoid false alarms from shot noise while catching real changes.

### 7. Baselines and Verification

**Baselines** are reference runs that represent "known good" behavior. They're essential for:

- **CI/CD pipelines**: Verify new code doesn't break experiments
- **Calibration monitoring**: Detect hardware drift
- **Regression testing**: Ensure reproducibility

#### Workflow:

1. Run experiment → get good results
2. Set that run as the **baseline** for the project
3. Later, run a **candidate** (new code, different time, etc.)
4. **Verify** the candidate against the baseline

#### VerifyPolicy options:

| Option | Purpose |
|--------|--------|
| `params_must_match` | Require identical parameters |
| `program_must_match` | Require identical circuit/program |
| `tvd_max` | Maximum allowed distribution distance |
| `allow_missing_baseline` | Pass if no baseline exists |

In [ ]:
# Set our first run as the baseline for this project
set_baseline(PROJECT, first_run_id)

baseline_info = get_baseline(PROJECT)
print(f"Baseline set for project '{PROJECT}'")
print(f"  Run ID: {baseline_info['run_id']}")

In [ ]:
# Verify candidate with STRICT policy (params must match)
strict_policy = VerifyPolicy(
    params_must_match=True,  # Require same parameters
    program_must_match=False,  # No circuit in this demo
    tvd_max=0.10,
)

result = verify_baseline(candidate_id, project=PROJECT, policy=strict_policy)

print("Strict Verification (params_must_match=True)")
print("=" * 50)
print(f"Result: {'[OK] PASSED' if result.ok else '[X] FAILED'}")

if not result.ok:
    print("\nFailures:")
    for failure in result.failures:
        print(f"  {failure}")

In [ ]:
# Verify candidate with RELAXED policy (params can differ)
relaxed_policy = VerifyPolicy(
    params_must_match=False,  # Allow different parameters
    program_must_match=False,
    tvd_max=0.10,
)

result = verify_baseline(candidate_id, project=PROJECT, policy=relaxed_policy)

print("Relaxed Verification (params_must_match=False)")
print("=" * 50)
print(f"Result: {'[OK] PASSED' if result.ok else '[X] FAILED'}")

### 8. Bundles: Portable Run Sharing

**Bundles** are self-contained ZIP files that package a run with all its artifacts.

#### Use cases:

- **Share with collaborators**: Email a bundle, they can unpack and inspect
- **Attach to papers**: Include bundle as supplementary material
- **Archive experiments**: Long-term storage with all artifacts
- **Transfer between systems**: Move runs between workspaces

#### Bundle contents:

```
experiment.devqubit.zip
├── manifest.json       # Bundle metadata
├── run.json            # Run record (params, metrics, tags)
└── objects/            # Content-addressed artifacts
    ├── sha256:abc123...
    ├── sha256:def456...
    └── ...
```

In [ ]:
# Pack the baseline run into a bundle
bundle_path = WORKSPACE / "baseline_run.devqubit.zip"

pack_result = pack_run(first_run_id, bundle_path)

print("Bundle Created")
print("=" * 40)
print(f"Path:      {bundle_path}")
print(f"Size:      {bundle_path.stat().st_size / 1024:.1f} KB")
print(f"Artifacts: {pack_result.artifact_count}")
print(f"Objects:   {pack_result.object_count}")

In [ ]:
# Inspect bundle without extracting
with Bundle(bundle_path) as bundle:
    print("Bundle Contents")
    print("=" * 40)
    print(f"Run ID:  {bundle.run_id}")
    print(f"Project: {bundle.get_project()}")
    print(f"Objects: {len(bundle.list_objects)}")

### Unpacking Bundles

When someone receives a bundle, they can unpack it into their own workspace.

The run gets imported with:
- Same run ID (preserved for reference)
- All parameters, metrics, tags
- All artifacts (stored in their object store)

In [ ]:
# Create a new workspace (simulating a collaborator's machine)
collaborator_workspace = WORKSPACE / "collaborator"
collab_config = Config(root_dir=collaborator_workspace)
collab_store = create_store(config=collab_config)
collab_registry = create_registry(config=collab_config)

# Unpack the bundle
unpack_result = unpack_bundle(
    bundle_path,
    dest_store=collab_store,
    dest_registry=collab_registry,
)

print("Bundle Unpacked")
print("=" * 40)
print(f"Workspace: {collaborator_workspace}")
print(f"Run ID:    {unpack_result.run_id}")
print(f"Artifacts: {unpack_result.artifact_count}")

In [ ]:
# Verify the unpacked run matches the original
original = load_run(first_run_id)
imported = collab_registry.load(unpack_result.run_id)

print("Integrity Check")
print("=" * 40)
print(f"Run IDs match: {original.run_id == imported.run_id}")
print(f"Params match:  {original.params == imported.params}")
print(f"Metrics match: {original.metrics == imported.metrics}")
print(f"Tags match:    {original.tags == imported.tags}")

### 9. Backend Wrapping

When working with real quantum backends (Qiskit, Cirq, PennyLane, Braket), devqubit can **wrap** the backend to automatically capture:

- Executed circuits
- Measurement counts
- Job metadata

```python
from qiskit_aer import AerSimulator

with track(project="quantum_experiment") as run:
    # Wrap the backend — devqubit intercepts all circuit executions
    backend = run.wrap(AerSimulator())
    
    # Use the wrapped backend normally
    job = backend.run(circuit, shots=1000)
    counts = job.result().get_counts()
    
    # Circuit and counts are automatically stored as artifacts!
```

See the SDK-specific notebooks (01-05) for complete examples.

### 9. What Happens Under the Hood

When you use devqubit, several things happen automatically. Understanding this helps you use it more effectively.

#### When you call `track()`:

There are two paths depending on whether you use an adapter:

**Path A: With adapter (`run.wrap()`)**
```python
with track(project="my_project") as run:
    run.log_params({...})               # Params stored in memory
    
    backend = run.wrap(AerSimulator())  # Adapter wraps backend
    job = backend.run(circuit)          # Adapter captures: circuit, device, options
    counts = job.result().get_counts()  # Adapter captures: results, timestamps
    
# On exit:
# → Adapter builds FULL ExecutionEnvelope:
#   - producer (adapter name, SDK version)
#   - program (logical + physical circuits, hashes)
#   - device (backend, qubits, calibration)
#   - execution (shots, job_ids, timestamps)
#   - result (counts, success status)
# → Envelope validated against UEC schema
# → Envelope + artifacts stored in Object Store
# → Run record finalized in Registry
```

**Path B: Manual tracking (no adapter)**
```python
with track(project="my_project") as run:
    run.log_params({...})
    run.log_metric(...)
    run.log_json("my_results", {...})   # Your artifacts
    
    # No run.wrap() — you manage execution yourself
    
# On exit:
# → Engine synthesizes MINIMAL ExecutionEnvelope:
#   - producer (adapter: "manual")
#   - result (success: true/false based on exceptions)
#   - NO program/device/execution snapshots
# → Your artifacts stored in Object Store
# → Run record finalized in Registry
```

**Recommendation:** Use `run.wrap()` when possible — it gives devqubit everything needed for robust comparison and verification.

#### Content-Addressed Storage

All artifacts (including ExecutionEnvelopes) are stored by their **SHA-256 hash**:
```
objects/
├── sha256:a1b2c3...  ← ExecutionEnvelope JSON
├── sha256:d4e5f6...  ← circuit.qpy (referenced by envelope)
├── sha256:g7h8i9...  ← raw_result.json (referenced by envelope)
```

**Benefits:**
- **Deduplication**: Same circuit run twice → stored once
- **Integrity**: Hash verifies content hasn't changed
- **Portability**: Bundles include only referenced objects
- **Fast comparison**: Compare hashes instead of full content

#### How Fingerprints Work

ExecutionEnvelope contains pre-computed hashes for fast comparison:

| Fingerprint | Source | Use Case |
|-------------|--------|----------|
| `structural_hash` | Circuit structure (gates, qubits) | "Is this the same algorithm?" |
| `parametric_hash` | Structure + bound parameters | "Is this the exact same circuit?" |
| `executed_structural_hash` | Transpiled circuit structure | "Did transpilation change?" |

These enable `diff()` and `verify_baseline()` to compare runs without loading full circuit data.

### Next Steps

Explore the SDK-specific notebooks:

| Notebook | SDK | Focus |
|----------|-----|-------|
| **01_vqe_optimization_qiskit** | Qiskit | VQE with sweeps and verification |
| **02_qml_classification_pennylane** | PennyLane | QML training with epoch metrics |
| **03_noise_analysis_cirq** | Cirq | Noise model comparison |
| **04_bundle_sharing_braket** | Braket | Cross-workspace bundle workflow |
| **05_drift_detection_runtime** | Runtime | Production monitoring with CI/CD |

In [ ]:
# Clean up demo workspace
shutil.rmtree(WORKSPACE)
print("Demo workspace cleaned up")